# Import Dependencies

In [ ]:
import whisper
from translate import Translator
import datetime

# Transcribe

In [ ]:
model = whisper.load_model('base')
result  =  model.transcribe('interview_1_17_24.mp4')

In [ ]:
for index, segment in enumerate(result['segments']):
    print(segment)

In [ ]:
transcribe_file = 'transcribe_file.txt'
translate_file = 'translate_file.srt'

# Translate

In [ ]:
with open(transcribe_file,'w') as file:
    for index, segment in enumerate(result['segments']):
        file.write(str(index+1)+'\n')
        file.write(str(datetime.timedelta(seconds=segment['start'])) + ',000'  + ' --> ' + str(datetime.timedelta(seconds=segment['end']))  + ',000' +'\n')
        # file.write(str(datetime.timedelta(seconds=segment['start']) + datetime.timedelta(0,836)) + ',000'  + ' --> ' + str(datetime.timedelta(seconds=segment['end']) + datetime.timedelta(0,836))  + ',000' +'\n')
        file.write(segment['text'].strip()+'\n')
        file.write('\n')

In [ ]:
text = '老人'
tran = Translator(from_lang="zh",to_lang='en')
translation = tran.translate(text)
print(f'{translation}')

In [ ]:
tran = Translator(from_lang="zh-TW",to_lang='en')
count = 2
with open(translate_file,'w') as english_file:
    with open(transcribe_file,'r') as chinese_file:
        lines = chinese_file.readlines()
        for index, segment in enumerate(result['segments']):
            print(lines[count-1])
            # print(lines[count])
            line_to_translate = lines[count]
            # print(index)
            translation = tran.translate(line_to_translate)
            # print(translation)
            english_file.write(str(index+1)+'\n')
            english_file.write(lines[count-1]+'\n')
            # english_file.write(lines[count-1][:7] + ',000 ' + lines[count-1][7:10] + ' ' + lines[count-1][10:17] + ',000 '+'\n')
            english_file.write(translation.strip()+'\n')
            english_file.write('\n')
            count += 4
            # if count < 10:
            #     continue
            # else:
            #     break

# Add subtitles to video

In [ ]:
import sys
import pysrt
from moviepy.editor import VideoFileClip, TextClip, CompositeVideoClip


def time_to_seconds(time_obj):
    return time_obj.hours * 3600 + time_obj.minutes * 60 + time_obj.seconds + time_obj.milliseconds / 1000


def create_subtitle_clips(subtitles, videosize,fontsize=24, font='Arial', color='yellow', debug = False):
    subtitle_clips = []

    for subtitle in subtitles:
        # print(subtitle)
        start_time = time_to_seconds(subtitle.start)
        end_time = time_to_seconds(subtitle.end)
        duration = end_time - start_time

        video_width, video_height = videosize
        
        text_clip = TextClip(subtitle.text, fontsize=fontsize, font=font, color=color, bg_color = 'black',size=(video_width*3/4, None), method='caption').set_start(start_time).set_duration(duration)
        subtitle_x_position = 'center'
        subtitle_y_position = video_height* 4 / 5 

        text_position = (subtitle_x_position, subtitle_y_position)                    
        subtitle_clips.append(text_clip.set_position(text_position))

    return subtitle_clips



In [ ]:
# Load video and SRT file
video = VideoFileClip(video_path)
subtitles = pysrt.open(srt_path)

begin,end= video_path.split(".mp4")
output_video_file = begin+'_subtitled'+".mp4"

print ("Output file name: ",output_video_file)

# Create subtitle clips
subtitle_clips = create_subtitle_clips(subtitles,video.size)

# Add subtitles to the video
final_video = CompositeVideoClip([video] + subtitle_clips)

# Write output video file
final_video.write_videofile(output_video_file)
